# Remove unpublished blueprints

In [1]:
import sqlite3
from monad_std import *

In [2]:
db_blp = sqlite3.connect('blueprints.db')
cur_blp = db_blp.cursor()
db_name = sqlite3.connect('id-name.db')
cur_name = db_name.cursor()

In [3]:
def is_published(type_id: int) -> bool:
    cur_name.execute("select published from typename where type_id == ?", (type_id,))
    val = cur_name.fetchone()
    return val is not None and val[0] == 1

In [4]:
cur_blp.execute("select type_id from blp_name;")
blp_ids = [x[0] for x in cur_blp.fetchall()]

In [5]:
id_to_delete = []
for blp_id in blp_ids:
    if not is_published(blp_id):
        id_to_delete.append(blp_id)

In [6]:
cur_blp.executemany("delete from blp_name where type_id == ?", map(lambda x: (x,), id_to_delete))
cur_blp.executemany("delete from blp_recursive where blp_id == ?", map(lambda x: (x,), id_to_delete))
cur_blp.executemany("delete from material where type_id == ?", map(lambda x: (x,), id_to_delete))
cur_blp.executemany("delete from product where blp_id == ?", map(lambda x: (x,), id_to_delete))
cur_blp.executemany("delete from product where product_id == ?", map(lambda x: (x,), id_to_delete))
db_blp.commit()